OIB radar reduction

Notebook to take block averges of radar derived thickness and bed elevation to reduce data volume

In [1]:
### imports ##
import os, sys, math
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pyproj
import verde as vd

In [2]:
dat_path = 'C:/Users/btober/OneDrive/Documents/MARS/orig/xtra/OIB-AK/radar/'

In [3]:
gdf = gpd.read_file(dat_path + 'OIB-AK_pk_bst.gpkg')

In [4]:
gdf.head()

,track,trace,lon,lat,elev,srf_sample,srf_twtt,srf_elev,srf_amp,bed_sample,bed_twtt,bed_amp,bed_elev,srf_bed_thick,geometry
0,IRUAFHF1B_20130322-205751,0,-142.755324,60.889714,1660.484070,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-142.75532 60.88971)
1,IRUAFHF1B_20130322-205751,1,-142.755481,60.889310,1658.535081,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-142.75548 60.88931)
2,IRUAFHF1B_20130322-205751,2,-142.755557,60.889108,1657.981002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-142.75556 60.88911)
3,IRUAFHF1B_20130322-205751,3,-142.755634,60.888906,1657.426938,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-142.75563 60.88891)
4,IRUAFHF1B_20130322-205751,4,-142.755711,60.888704,1656.625443,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-142.75571 60.88870)


In [5]:
# project
gdf = gdf.to_crs('epsg:3338')

In [6]:
# use verde to reduce along track data 
reducer = vd.BlockReduce(np.mean, spacing=50)

track = []
x = []
y = []
srf_z = []
bed_z = []
h = []

for track_ in list(gdf.track.unique()):
    (x_, y_), srf_ = reducer.filter((gdf[gdf['track']==track_].centroid.x.to_numpy(), gdf[gdf['track']==track_].centroid.y.to_numpy()), gdf[gdf['track']==track_].srf_elev.to_numpy())
    (x_, y_), bed_ = reducer.filter((gdf[gdf['track']==track_].centroid.x.to_numpy(), gdf[gdf['track']==track_].centroid.y.to_numpy()), gdf[gdf['track']==track_].bed_elev.to_numpy())
    (x_, y_), h_ = reducer.filter((gdf[gdf['track']==track_].centroid.x.to_numpy(), gdf[gdf['track']==track_].centroid.y.to_numpy()), gdf[gdf['track']==track_].srf_bed_thick.to_numpy())
    x.append(x_)
    y.append(y_)
    srf_z.append(srf_)
    bed_z.append(bed_)
    h.append(h_)
    track.append([track_ for i in range(len(x_))])

track = np.hstack(track)
x = np.hstack(x)
y = np.hstack(y)
srf_z = np.hstack(srf_z)
bed_z = np.hstack(bed_z)
h = np.hstack(h)

In [8]:
df = pd.DataFrame(
    {'track': track,
     'srf_z': srf_z,
     'bed_z': bed_z,
     'h': h,
     'x': x,
     'y': y})
     
gdf_out = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.x, df.y), crs='epsg:3338').to_crs('epsg:4326')

gdf_out = gdf_out.drop(columns=['x','y'])

gdf_out.to_file(dat_path + 'OIB-AK_pk_bst_block_reduce.gpkg', driver="GPKG")